In [ ]:
# default_exp core

# pico_pi_controller

> A service interface for any Raspberry Pi like SBC to be monitored & controlled from a CircuitPython capable microcontroller.

In [ ]:
#hide
from nbdev.showdoc import *

This module also distributes the CircuitPython `code.py` file for a Raspberry Pi Pico configured as shown in the instructions below. 

In [ ]:
#export
def connect_mcu():
    return True

A fake test to trigger GitHub Actions CI

In [ ]:
c = connect_mcu()
assert str(c) == 'True'